<a href="https://colab.research.google.com/github/sajidcsecu/radioGenomic/blob/main/3dunetingpu__nifti_monai6_1_3_balanced.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This is the Code for the Segmentation on Rider Dataset (LUNG1). The Code is worked on the 3D volume over GPU. This is the improved of Model 5
1. the architecture is 3DUNet
2. The balanced sampler, preprocessed data (uniform volume spacing and clipping [-1000, 700]) and the
3. strong augmentation is used in the code...
4. Dice Loss is 1 and Binary classification Entropy is 1
5. Number of positive pixels is for all patients

# (1) Import Required Libraries

In [1]:
!pip install SimpleITK
!pip install pydicom===2.4.3
!pip install pydicom-seg
!pip install numpy==1.23.5
!pip install monai
!pip install torch==1.13.1
!pip install nibabel>=5.0.0

  Using cached numpy-1.23.5.tar.gz (10.7 MB)
  Installing build dependencies ... done
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Getting requirements to build wheel ... error
error: subprocess-exited-with-error

× Getting requirements to build wheel did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 86.5 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement torch==1.13.1 (from versions: 2.2.0, 2.2.1, 2.2.2, 2.3.0, 2.3.1, 2.4.0, 2.4.1, 2.5.0, 2.5.1, 2.6.0, 2.7.0, 2.7.1, 2.8.0, 2.9.0)
ERROR: No matching distribution found for torch==1.13.1



# (2) Import required Libraries

In [2]:
import os
from pathlib import Path
import csv
import time
import random
import shutil
from glob import glob
from typing import List
import torch
import numpy as np
import torch.nn as nn
import torch.cuda.amp as amp
from torch.optim import lr_scheduler
from monai.inferers import sliding_window_inference
from monai.transforms import AsDiscrete
from monai.networks.nets import UNet
from monai.transforms import (
    Compose,
    LoadImaged,
    EnsureChannelFirstd,
    Spacingd,
    Orientationd,
    ScaleIntensityRanged,
    CropForegroundd,
    ResizeWithPadOrCropd,
    RandCropByPosNegLabeld,
    RandFlipd,
    RandAffined,
    RandGaussianNoised,
    RandScaleIntensityd,
    ToTensord,
    EnsureTyped,
    EnsureChannelFirstD,
    SpatialPadd,
    Rand3DElasticd,
    NormalizeIntensityd,
    RandGaussianSmoothd,
    RandAdjustContrastd,
    AsDiscreted,
    Lambdad,
    CenterSpatialCropd

)
from monai.transforms import MapTransform
from monai.utils import ensure_tuple_rep
import json
from monai.data import CacheDataset
from monai.transforms import Transform
from monai.data import CacheDataset
import torch.nn.functional as F
from monai.data import Dataset, DataLoader, CacheDataset, pad_list_data_collate
from monai.networks.layers import Norm
import nibabel as nib
from sklearn.metrics import jaccard_score, f1_score, recall_score, precision_score, accuracy_score
import matplotlib.pyplot as plt
import multiprocessing as mp
from typing import Dict, List, Tuple, Optional
from monai.transforms import EnsureTyped
from monai.transforms import SaveImaged
from monai.utils import set_determinism
from monai.metrics import DiceMetric, HausdorffDistanceMetric, SurfaceDistanceMetric
import warnings
warnings.filterwarnings("ignore", message="Num foregrounds 0, Num backgrounds.*unable to generate class balanced samples")

# (3) Mount Google Drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Safe Unet

In [4]:
class SafeUNet(torch.nn.Module):
    def __init__(self, base_unet):
        super().__init__()
        self.unet = base_unet

    def forward(self, x):
        try:
            # Try normal forward pass
            return self.unet(x)
        except RuntimeError as e:
            if "Sizes of tensors must match" in str(e):
                print(f"⚠️ Size mismatch detected! Input: {x.shape}")
                print("🔄 Attempting automatic resize...")

                # Emergency resize to compatible size
                compatible_size = self.find_compatible_size(x.shape[2:])
                x_resized = F.interpolate(
                    x,
                    size=compatible_size,
                    mode='trilinear',
                    align_corners=False
                )

                # Forward pass with resized input
                output = self.unet(x_resized)

                # Resize output back to original size
                output = F.interpolate(
                    output,
                    size=x.shape[2:],
                    mode='trilinear',
                    align_corners=False
                )
                print(f"✅ Resize successful: {x.shape[2:]} → {compatible_size} → {x.shape[2:]}")
                return output
            else:
                raise e

    def find_compatible_size(self, original_size):
        """Find a size compatible with UNet architecture (divisible by 16)"""
        # UNet needs sizes divisible by 2^num_downsampling (16 for 4 downsamples)
        compatible_size = tuple((s // 16) * 16 for s in original_size)

        # Ensure minimum size
        compatible_size = tuple(max(16, s) for s in compatible_size)

        print(f"🎯 Original: {original_size}, Compatible: {compatible_size}")
        return compatible_size

## (4). Loss Function

In [5]:
class ImprovedDiceFocalLoss(nn.Module):
    # Definition stub for runnable code (replace with your actual definition)
    def __init__(self, smooth=1e-6, dice_weight=0.7, focal_weight=0.3, gamma=2.0, alpha=0.25, pos_weight=None):
        super().__init__()
        self.smooth = smooth
        self.dice_weight = dice_weight
        self.focal_weight = focal_weight
        self.gamma = gamma
        self.alpha = alpha
        self.pos_weight = pos_weight

    def forward(self, preds, targets):
        preds_sigmoid = torch.sigmoid(preds)

        # Dice Loss
        intersection = (preds_sigmoid * targets).sum()
        dice_loss = 1 - (2. * intersection + self.smooth) / (
            preds_sigmoid.sum() + targets.sum() + self.smooth
        )

        # Focal Loss with pos_weight
        # NOTE: F.binary_cross_entropy_with_logits handles pos_weight internally
        if self.pos_weight is not None:
            bce_loss = F.binary_cross_entropy_with_logits(
                preds, targets,
                reduction='none',
                pos_weight=self.pos_weight
            )
        else:
            bce_loss = F.binary_cross_entropy_with_logits(preds, targets, reduction='none')

        pt = torch.exp(-bce_loss)
        focal_loss = self.alpha * (1-pt)**self.gamma * bce_loss

        return self.dice_weight * dice_loss + self.focal_weight * focal_loss.mean()

# (5). Test

In [6]:
class UnetTest:
    def __init__(self, test_result_path: str, metrics_csv: str, device: torch.device):
        # Remove super().__init__() since there's no parent class
        self.test_result_path = test_result_path
        self.metrics_csv = metrics_csv
        self.device = device

        self.dice_metric = DiceMetric(include_background=False, reduction="mean")
        self.hd95_metric = HausdorffDistanceMetric(include_background=False, percentile=95, reduction="mean")
        self.asd_metric = SurfaceDistanceMetric(include_background=False, reduction="mean")

        # Create test output directory
        os.makedirs(self.test_result_path, exist_ok=True)

        # Update CSV headers to include new metrics
        self._init_enhanced_metrics_csv()

    def _init_enhanced_metrics_csv(self):
        """Initialize CSV with additional metric columns"""
        if not os.path.exists(self.metrics_csv):
            with open(self.metrics_csv, 'w', newline='') as f:
                writer = csv.writer(f)
                writer.writerow([
                    "SampleID", "Jaccard", "F1", "Recall", "Precision", "Accuracy",
                    "Dice", "HD95", "ASD", "Time"
                ])

    def calculate_basic_metrics(self, y_true: np.ndarray, y_pred: np.ndarray):
        """Calculate basic segmentation metrics (replacement for missing parent class)"""
        # If you had a parent class with calculate_metrics, implement it here
        eps = 1e-8

        tp = np.sum(y_true * y_pred)
        fp = np.sum((1 - y_true) * y_pred)
        fn = np.sum(y_true * (1 - y_pred))
        tn = np.sum((1 - y_true) * (1 - y_pred))

        jaccard = tp / (tp + fp + fn + eps)
        f1 = 2 * tp / (2 * tp + fp + fn + eps)
        recall = tp / (tp + fn + eps)
        precision = tp / (tp + fp + eps)
        accuracy = (tp + tn) / (tp + tn + fp + fn + eps)

        return [jaccard, f1, recall, precision, accuracy]

    def calculate_comprehensive_metrics(self, y_true: np.ndarray, y_pred: np.ndarray):
        """Calculate both basic and medical image metrics"""
        # Use the new basic metrics method
        basic_metrics = self.calculate_basic_metrics(y_true, y_pred)

        # Convert to torch tensors for MONAI metrics
        y_true_t = torch.from_numpy(y_true.astype(np.float32)).unsqueeze(0).unsqueeze(0)
        y_pred_t = torch.from_numpy(y_pred.astype(np.float32)).unsqueeze(0).unsqueeze(0)

        # Calculate MONAI metrics
        dice_value = self.dice_metric(y_pred_t, y_true_t)
        hd95_value = self.hd95_metric(y_pred_t, y_true_t)
        asd_value = self.asd_metric(y_pred_t, y_true_t)

        # Reset metrics for next calculation
        self.dice_metric.reset()
        self.hd95_metric.reset()
        self.asd_metric.reset()

        return basic_metrics + [
            dice_value.item() if not dice_value.isnan() else 0.0,
            hd95_value.item() if not hd95_value.isnan() else 0.0,
            asd_value.item() if not asd_value.isnan() else 0.0
        ]

    def save_result_slices(self, image_np: np.ndarray, pred_np: np.ndarray, label_np: np.ndarray, sample_id: str):
        """Save sample slices for visualization"""
        try:
            # Find slices with predictions for better visualization
            pred_slices = np.where(np.any(pred_np, axis=(0, 1)))[0]
            label_slices = np.where(np.any(label_np, axis=(0, 1)))[0]

            # Combine and get unique slices of interest
            slices_of_interest = np.unique(np.concatenate([pred_slices, label_slices]))

            # If no positive slices, use center slices
            if len(slices_of_interest) == 0:
                slices_of_interest = [pred_np.shape[2] // 2 - 1, pred_np.shape[2] // 2, pred_np.shape[2] // 2 + 1]

            # Save a few representative slices
            for i, slice_idx in enumerate(slices_of_interest[:3]):  # Save max 3 slices
                if 0 <= slice_idx < pred_np.shape[2]:
                    # Create a simple visualization
                    fig, axes = plt.subplots(1, 3, figsize=(12, 4))

                    axes[0].imshow(image_np[:, :, slice_idx], cmap='gray')
                    axes[0].set_title(f'Image - Slice {slice_idx}')
                    axes[0].axis('off')

                    axes[1].imshow(label_np[:, :, slice_idx], cmap='jet', alpha=0.7)
                    axes[1].set_title('Ground Truth')
                    axes[1].axis('off')

                    axes[2].imshow(pred_np[:, :, slice_idx], cmap='jet', alpha=0.7)
                    axes[2].set_title('Prediction')
                    axes[2].axis('off')

                    plt.tight_layout()
                    save_path = os.path.join(self.test_result_path, f"{sample_id}_slice_{slice_idx}.png")
                    plt.savefig(save_path, dpi=100, bbox_inches='tight')
                    plt.close()

        except Exception as e:
            print(f"⚠️ Could not save slices for {sample_id}: {e}")

    def append_metrics_to_csv(self, sample_id: str, metrics: list, elapsed_time: float):
        """Override to handle extended metrics"""
        with open(self.metrics_csv, 'a', newline='') as f:
            writer = csv.writer(f)
            # Now metrics should have: [jaccard, f1, recall, precision, accuracy, dice, hd95, asd]
            writer.writerow([sample_id] + [f"{m:.4f}" for m in metrics] + [f"{elapsed_time:.4f}"])

    def test(self, model: nn.Module, test_loader: DataLoader):
        model.eval()
        total_metrics = np.zeros(8)  # Now 8 metrics total
        total_times = []

        # Count samples with actual predictions
        samples_with_predictions = 0

        roi_size = (96, 96, 96)
        sw_batch_size = 1

        with torch.no_grad():
            for batch_idx, batch in enumerate(test_loader):
                image, label = batch["vol"].to(self.device), batch["seg"].to(self.device)
                start_time = time.time()

                pred = sliding_window_inference(
                    inputs=image,
                    roi_size=roi_size,
                    sw_batch_size=sw_batch_size,
                    predictor=model
                )
                pred = torch.sigmoid(pred) > 0.5

                elapsed = time.time() - start_time
                total_times.append(elapsed)

                # Convert to NumPy
                image_np = image[0, 0].cpu().numpy()
                label_np = label[0, 0].cpu().numpy()
                pred_np = pred[0, 0].cpu().numpy().astype(np.uint8)

                # Skip if no predictions AND no ground truth (empty case)
                if pred_np.sum() == 0 and label_np.sum() == 0:
                    print(f"📝 Sample {batch_idx}: No predictions and no ground truth - skipping")
                    continue

                # Enhanced metrics calculation
                metrics = self.calculate_comprehensive_metrics(label_np, pred_np)
                total_metrics += np.array(metrics)
                samples_with_predictions += 1

                sample_id = f"sample_{batch_idx:03d}"
                self.save_result_slices(image_np, pred_np, label_np, sample_id)
                self.append_metrics_to_csv(sample_id, metrics, elapsed)

                # Print sample-level results
                print(f"📊 Sample {batch_idx}: Dice={metrics[5]:.4f}, HD95={metrics[6]:.2f}mm, Time={elapsed:.2f}s")

        # Print enhanced summary (only for samples with meaningful data)
        if samples_with_predictions > 0:
            num_samples = samples_with_predictions
            metric_names = ["Jaccard", "F1", "Recall", "Precision", "Accuracy", "Dice", "HD95", "ASD"]
            print("\n" + "="*60)
            print("📊 ENHANCED TEST METRICS SUMMARY:")
            print("="*60)
            for i, name in enumerate(metric_names):
                print(f"{name:<12}: {total_metrics[i]/num_samples:.4f}")
            print(f"📈 Samples evaluated: {num_samples}/{len(test_loader)}")
            print(f"⚡ Average FPS: {1 / np.mean(total_times):.2f}")
            print(f"⏱️  Average time per sample: {np.mean(total_times):.2f}s")
        else:
            print("❌ No samples with meaningful predictions to evaluate!")

## (6) Early Stopping

In [7]:
class EarlyStopping:
    def __init__(self, patience=10, verbose=True, min_delta=0, path='checkpoint.pt',
                 start_val_loss_min=None, start_patience_counter=0):
        self.patience = patience
        self.verbose = verbose
        self.min_delta = min_delta
        self.path = path
        self.val_loss_min = start_val_loss_min if start_val_loss_min is not None else np.inf
        self.counter = start_patience_counter
        self.early_stop = False

    def __call__(self, val_loss, model, epoch=None, optimizer=None):
        improved = False
        if val_loss < self.val_loss_min - self.min_delta:
            self.val_loss_min = val_loss
            self.counter = 0
            improved = True
            if self.verbose:
                print(f"✅ Validation loss improved ({self.val_loss_min:.6f}). Saving model...")
        else:
            self.counter += 1
            if self.verbose:
                print(f"⏳ EarlyStopping counter: {self.counter} out of {self.patience}")

        # Always save a full checkpoint
        self.save_checkpoint(model, epoch, optimizer)

        if self.counter >= self.patience:
            self.early_stop = True

        return self.early_stop

    def save_checkpoint(self, model, epoch=None, optimizer=None):
        checkpoint = {
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict() if optimizer else None,
            'val_loss': self.val_loss_min,
            'patience_counter': self.counter,
        }
        torch.save(checkpoint, self.path)
        if self.verbose:
            print(f"💾 Checkpoint saved.")

# (7). Training

In [8]:

# ---------- Trainer ----------

class UnetTrain:
    def __init__(self, model_file, loss_result_path, lr, num_epochs, device):
        self.model_file = model_file
        self.loss_result_path = loss_result_path
        self.lr = lr
        self.num_epochs = num_epochs
        self.device = device
        # Initialize pos_weight_file: This is the dedicated JSON path.
        self.pos_weight_file = model_file.replace('.pth', '_pos_weight.json')
        self.seeding(42)

    def seeding(self, seed):
        random.seed(seed)
        np.random.seed(seed)
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

    def epoch_time(self, start_time, end_time):
        elapsed = end_time - start_time
        return int(elapsed / 60), int(elapsed % 60)

    def train_one_epoch(self, model, loader, optimizer, accumulation_steps, loss_fn):
        model.train()
        epoch_loss = 0
        # Uses mixed-precision training (amp) for efficiency
        scaler = torch.amp.GradScaler()
        device_type = 'cuda' if self.device.type == 'cuda' else 'cpu'

        positive_patches = 0
        total_patches = 0

        optimizer.zero_grad()

        # FIX: Calculate the total number of iterations (patches) for accurate reporting
        # Assuming loader yields 8 patches per original volume (num_samples=8)
        total_iterations = len(loader) * 8

        for i, batch in enumerate(loader):
            # 1. Load data to device
            inputs, labels = batch["vol"].to(self.device, non_blocking=True), batch["seg"].to(self.device, non_blocking=True)

            # 2. Monitor patch quality (checking if the current batch contains a positive lesion)
            batch_positives = labels.sum().item()
            if batch_positives > 0:
                positive_patches += 1
            total_patches += 1 # Total number of batches processed

            # 3. Forward Pass and Loss Calculation (using automatic mixed precision)
            with torch.amp.autocast(device_type=device_type, enabled=True):
                outputs = model(inputs)
                # Loss is scaled down by accumulation_steps for averaging
                loss = loss_fn(outputs, labels) / accumulation_steps

            # 4. Backward Pass (Gradient calculation)
            scaler.scale(loss).backward()

            # 5. Gradient Accumulation Check (Updates weights only every N steps)
            if (i + 1) % accumulation_steps == 0:
                scaler.step(optimizer) # Apply accumulated gradients
                scaler.update()        # Update the scaler for mixed precision
                optimizer.zero_grad()  # Clear gradients for the next cycle

            # 6. Accumulate loss for reporting (rescale loss back up)
            epoch_loss += loss.item() * accumulation_steps

            # 7. Progress reporting (using the corrected total_iterations)
            if (i + 1) % 50 == 0:
                print(f"  Batch {i+1}/{total_iterations}, Loss: {loss.item() * accumulation_steps:.4f}")

        # Final print statement for Patch Quality
        print(f"  Patch Quality: {positive_patches}/{total_patches} ({positive_patches/total_patches*100:.1f}%) with positives")

        # Return the final averaged loss over the true number of iterations
        return epoch_loss / total_iterations


    def evaluate(self, model, loader, loss_fn):
        model.eval()
        epoch_loss = 0
        with torch.no_grad():
            for batch in loader:
                inputs, labels = batch["vol"].to(self.device), batch["seg"].to(self.device)
                outputs = model(inputs)
                loss = loss_fn(outputs, labels)
                epoch_loss += loss.item()
        return epoch_loss / len(loader)

    def save_pos_weight(self, pos_weight):
        """Save pos_weight to JSON file"""
        with open(self.pos_weight_file, 'w') as f:
            json.dump({'pos_weight': pos_weight, 'timestamp': time.time()}, f)
        print(f"💾 Saved pos_weight: {pos_weight:.4f}")

    def load_pos_weight(self):
        """Load pos_weight from JSON file"""
        # Condition 1 & 2: Check if file exists and read it
        if os.path.exists(self.pos_weight_file):
            try:
                with open(self.pos_weight_file, 'r') as f:
                    data = json.load(f)
                    pos_weight = data['pos_weight']
                    print(f"📂 Loaded pos_weight: {pos_weight:.4f}")
                    return pos_weight
            except Exception as e:
                print(f"⚠️ Error loading pos_weight: {e}")
        return None

    def get_pos_weight(self, train_loader, force_recompute=False):
        """Get pos_weight - load if exists, otherwise compute and save"""
        if not force_recompute:
            pos_weight = self.load_pos_weight()
            if pos_weight is not None:
                return pos_weight

        # Condition 3: Compute new pos_weight and save it
        pos_weight = self.compute_pos_weight(train_loader, max_batches=None)
        self.save_pos_weight(pos_weight)
        return pos_weight

    def compute_pos_weight(self, train_loader, max_batches=None):
        """Compute pos_weight with better diagnostics"""
        pos, neg = 0, 0
        total_samples = 0
        total_batch_pos = 0

        for i, batch in enumerate(train_loader):
            seg = batch["seg"].to(self.device)
            batch_pos = seg.sum().item()
            batch_neg = seg.numel() - batch_pos

            pos += batch_pos
            neg += batch_neg
            total_samples += 1

            print(f"Batch {i}: pos={batch_pos}, neg={batch_neg}, pos_ratio={batch_pos/seg.numel():.6f}")

            if(batch_pos > 0):
                total_batch_pos += 1

            if max_batches and (i + 1) >= max_batches:
                break



        pos_ratio = pos / (pos + neg) if (pos + neg) > 0 else 0
        pos_weight = neg / (pos + 1e-8)

        print(f"📊 Final Stats:")
        print(f"  Total pos voxels: {pos}")
        print(f"  Total neg voxels: {neg}")
        print(f"  Positive ratio: {pos_ratio:.6f} ({pos_ratio*100:.4f}%)")
        print(f"  Computed pos_weight: {pos_weight:.4f}")
        print(f"  Total batches with positives: {total_batch_pos}")
        return pos_weight

    def execute(self, model, train_loader, valid_loader):
        # ... (Optimizer and Scheduler remain unchanged)
        optimizer = torch.optim.AdamW(model.parameters(), lr=self.lr, weight_decay=1e-4)

        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer,
            mode='min',
            factor=0.5,
            patience=10,
            min_lr=1e-6,
            threshold=0.0001
        )

        # 🔹 LINK: Auto-compute/load pos_weight before training (Handles all 3 file conditions)
        pos_weight = self.get_pos_weight(train_loader)
        # Cap pos_weight to reasonable value
        reasonable_pos_weight = min(pos_weight, 100.0)
        print(f"📊 Using reasonable pos_weight: {reasonable_pos_weight:.2f} (capped from {pos_weight:.2f})")

        # ✅ Initialize loss function using the final fixed weight
        loss_fn = ImprovedDiceFocalLoss(
            dice_weight=0.9,
            focal_weight=0.1,
            gamma=2.0,
            alpha=0.5,
            pos_weight=torch.tensor([reasonable_pos_weight], device=self.device)
        )

        accumulation_steps = 4

        # ---- Resume training state ----
        start_epoch = 1
        start_val_loss_min = None
        start_patience_counter = 0
        history = {"train_loss": [], "valid_loss": []}

        if os.path.exists(self.model_file):
            checkpoint = torch.load(self.model_file, map_location=self.device)
            model.load_state_dict(checkpoint['model_state_dict'])
            if checkpoint.get('optimizer_state_dict'):
                optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
            start_epoch = checkpoint.get('epoch', 1) + 1
            start_val_loss_min = checkpoint.get('val_loss', None)
            start_patience_counter = checkpoint.get('patience_counter', 0)

            # NOTE: Removed all checkpoint pos_weight loading logic, relying on self.get_pos_weight() above.

        # 🎯 Initialize EarlyStopping (pos_weight parameter is no longer required)
        early_stopping = EarlyStopping(
            patience=20,
            min_delta=0.0002,
            path=self.model_file,
            start_val_loss_min=start_val_loss_min,
            start_patience_counter=start_patience_counter,
        )

        if not os.path.exists(self.loss_result_path):
            with open(self.loss_result_path, "w", newline="") as f:
                csv.writer(f).writerow(["Epoch", "Train Loss", "Valid Loss"])

        # ---- Training loop ----
        for epoch in range(start_epoch, self.num_epochs + 1):
            start_time = time.time()

            train_loss = self.train_one_epoch(
                model, train_loader, optimizer, accumulation_steps, loss_fn
            )
            valid_loss = self.evaluate(model, valid_loader, loss_fn)
            scheduler.step(metrics=valid_loss)
            mins, secs = self.epoch_time(start_time, time.time())
            print(f"Epoch {epoch:03d} | Time: {mins}m {secs}s | "
                  f"Train: {train_loss:.6f} | Val: {valid_loss:.6f}")

            history['train_loss'].append(train_loss)
            history['valid_loss'].append(valid_loss)

            with open(self.loss_result_path, "a", newline="") as f:
                csv.writer(f).writerow([epoch, train_loss, valid_loss])

            # 🎯 Call EarlyStopping without pos_weight argument
            if early_stopping(valid_loss, model, epoch, optimizer):
                print("🛑 Early stopping triggered.")
                break

            torch.cuda.empty_cache()


In [9]:
class BinarizeSegd(MapTransform):
    """
    Custom MapTransform to explicitly binarize the segmentation label map.
    This replaces the unpicklable lambda function and the unavailable Thresholdd.
    """
    def __init__(self, keys, threshold=0.5, allow_missing_keys: bool = False):
        super().__init__(keys, allow_missing_keys)
        self.threshold = threshold

    def __call__(self, data):
        d = dict(data)
        for key in self.keys:
            # Ensure the segmentation tensor is float before comparison
            seg_tensor = d[key].to(torch.float32)

            # Binarize: x > threshold results in a boolean tensor (True/False)
            # .float() converts True to 1.0 and False to 0.0
            d[key] = (seg_tensor > self.threshold).float()

            # Crucial: Ensure the resulting segmentation is still 0 or 1 integers
            # if subsequent transforms or the UNet expect integer types for labels.
            # However, for loss calculation (BCELogits/Dice), float is usually fine.
            # Let's keep it float for compatibility with the UNet output and loss calculation,
            # but ensure the values are exactly 0.0 or 1.0.

        return d

In [10]:
# ----------------------------------------------------------------------
# GLOBAL HELPER FUNCTION (MUST be defined outside the UnetPipeline class)
# ----------------------------------------------------------------------
def binarize_segmentation_logic(data):
    """Binarizes the input tensor/array using a 0.5 threshold (Picklable)."""
    # This is the logic, and it MUST remain outside the class method.
    return (data > 0.5).float()

# (8). Pipeline

In [11]:
class UnetPipeline:
    def __init__(self, config: Dict):
        self.config = config
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        set_determinism(seed=42)

        # Setup paths
        os.chdir(self.config['target_dir'])
        print(f"📁 Current Directory: {os.getcwd()}")

        self.output_dir = os.path.join(".", "results", self.config['output_folder_name'])
        os.makedirs(self.output_dir, exist_ok=True)

        self.loss_result_file = os.path.join(self.output_dir, "train_and_valid_loss_results.csv")
        self.model_file = os.path.join(self.output_dir, "model.pth")
        self.test_metrics_file = os.path.join(self.output_dir, "test_metrics.csv")
        self.test_result_path = os.path.join(self.output_dir, "test_outputs")
        os.makedirs(self.test_result_path, exist_ok=True)

        self.dataset_dir = os.path.join("./datasets", f"Datasets_{self.config['transformation']}")

        # Prepare loaders
        print("📦 Preparing datasets and transforms...")
        self.train_loader, self.valid_loader, self.test_loader, self.full_train_loader = self.prepare_loaders()
        # self.debug_model_shapes()
        # Dataset quality analysis
        # print("\n" + "="*60)
        # print("🔍 RUNNING DATASET QUALITY ANALYSIS...")
        # print("="*60)
        # self.analyze_dataset_quality()

        # # 🔥 ADD COMPREHENSIVE SANITY CHECKS
        # print("\n" + "="*60)
        # print("✅ RUNNING COMPREHENSIVE DATA SANITY CHECKS")
        # print("="*60)
        # self.check_data_sanity(max_batches=2)
        # self.check_data_distributions()
        # self.check_spatial_consistency()


    def create_unet_model(self, device):
        """Create a robust UNet model that handles size variations."""

        # FIX: Only try to import UNet and Norm, which are typically present.
        try:
            from monai.networks.nets import UNet  # <--- ONLY import UNet
            from monai.networks.layers import Norm
        except ImportError as e:
            # Handle cases where MONAI is not correctly installed
            print(f"FATAL ERROR: Could not import MONAI base components. Check installation. Error: {e}")
            raise e

        # 1. Attempt DynamicUNet logic (now removed or commented out)

        print("⚠️ DynamicUNet not available or import failed. Using SafeUNet wrapper.")

        # 2. Use standard UNet wrapped by SafeUNet
        base_model = UNet(
            spatial_dims=3,
            in_channels=1,
            out_channels=1,
            # This configuration is based on standard UNet depth (5 layers, 4 strides)
            # It is the necessary minimum feature complexity, but extremely VRAM-intensive at 512^3.
            channels=(32, 64, 128, 256, 512),
            strides=(2, 2, 2, 2),
            num_res_units=2,
            norm=Norm.INSTANCE,
            dropout=0.1,
            act='PRELU'
        )
        # SafeUNet acts as the wrapper to catch runtime errors
        return SafeUNet(base_model).to(device)


    def analyze_dataset_quality(self):
        """Comprehensive dataset quality analysis"""
        print("📊 Analyzing dataset quality...")

        # Analyze full volumes (before patch sampling)
        self._analyze_loader_quality(self.full_train_loader, "TRAIN (Full Volumes)")
        self._analyze_loader_quality(self.valid_loader, "VALID (Full Volumes)")
        self._analyze_loader_quality(self.test_loader, "TEST (Full Volumes)")

        # Analyze training patches (after patch sampling)
        print("\n" + "🔍 Analyzing Training Patches (After Sampling):")
        self._analyze_patch_quality(self.train_loader, "TRAIN PATCHES")

    def debug_model_shapes(self):
        """Debug method to identify shape mismatches between model and data"""
        print("\n" + "="*60)
        print("🔍 DEBUGGING MODEL SHAPES")
        print("="*60)

        # Get a sample batch
        sample_batch = next(iter(self.train_loader))
        images, segs = sample_batch["vol"], sample_batch["seg"]

        # 🔥 FIX: Move data to the same device as model
        images = images.to(self.device)
        segs = segs.to(self.device)

        print(f"📐 Input data shape: {images.shape}")
        print(f"🎯 Target segmentation shape: {segs.shape}")
        print(f"💻 Input device: {images.device}")

        # NEW FIX: Clear cache before model creation
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            print("⚡️ Cleared CUDA cache before model instantiation.")

        # Test current model configuration


        current_model = self.create_unet_model(self.device)

        print(f"🤖 Model device: {next(current_model.parameters()).device}")

        with torch.no_grad():
            output = current_model(images)
            print(f"🤖 Current model output shape: {output.shape}")

            # Check for mismatch
            if output.shape[2:] != segs.shape[2:]:
                print(f"❌ SHAPE MISMATCH DETECTED!")
                print(f"   Model output: {output.shape[2:]}")
                print(f"   Target: {segs.shape[2:]}")
                print(f"   Difference: {tuple(o-t for o,t in zip(output.shape[2:], segs.shape[2:]))}")

                # 🔥 Calculate what the input size should be
                print(f"\n💡 SOLUTION: Your input size {images.shape[2:]} needs to be divisible by 16")
                print(f"   Try using: (96, 96, 96) → divisible by 16? {all(s % 16 == 0 for s in images.shape[2:])}")
            else:
                print("✅ Shapes match perfectly!")

        return output.shape, segs.shape


    def _analyze_loader_quality(self, loader, name: str):
        """Analyze a specific data loader"""
        print(f"\n📊 {name} Dataset Analysis:")
        print("-" * 40)

        total_volumes = 0
        volumes_with_positives = 0
        total_pos_voxels = 0
        total_voxels = 0

        for batch_idx, batch in enumerate(loader):
            seg = batch["seg"]

            # Check each volume in batch
            for i in range(seg.shape[0]):
                volume_seg = seg[i]
                total_volumes += 1
                volume_positives = volume_seg.sum().item()
                volume_voxels = volume_seg.numel()

                if volume_positives > 0:
                    volumes_with_positives += 1

                total_pos_voxels += volume_positives
                total_voxels += volume_voxels

        overall_pos_ratio = total_pos_voxels / total_voxels if total_voxels > 0 else 0

        print(f"  Total volumes: {total_volumes}")
        print(f"  Volumes with positives: {volumes_with_positives} ({volumes_with_positives/total_volumes*100:.1f}%)")
        print(f"  Overall positive ratio: {overall_pos_ratio:.6f} ({overall_pos_ratio*100:.4f}%)")
        print(f"  Total positive voxels: {total_pos_voxels:,}")
        print(f"  Total voxels: {total_voxels:,}")

        return overall_pos_ratio

    def _analyze_patch_quality(self, loader, name: str, max_batches: int = 20):
        """Analyze patch distribution in training loader"""
        print(f"\n📊 {name} Patch Analysis (first {max_batches} batches):")
        print("-" * 50)

        total_patches = 0
        patches_with_positives = 0
        total_pos_voxels = 0
        total_voxels = 0

        for batch_idx, batch in enumerate(loader):
            if batch_idx >= max_batches:
                break

            seg = batch["seg"]

            # Check each patch in batch
            for i in range(seg.shape[0]):
                patch_seg = seg[i]
                total_patches += 1
                patch_positives = patch_seg.sum().item()
                patch_voxels = patch_seg.numel()

                if patch_positives > 0:
                    patches_with_positives += 1

                total_pos_voxels += patch_positives
                total_voxels += patch_voxels

                pos_ratio = patch_positives / patch_voxels if patch_voxels > 0 else 0
                if patch_positives > 0:  # Only print patches with positives for clarity
                    print(f"  Batch {batch_idx}, Patch {i}: {patch_positives}/{patch_voxels} "
                          f"({pos_ratio*100:.3f}% positive)")

        overall_pos_ratio = total_pos_voxels / total_voxels if total_voxels > 0 else 0

        print(f"\n📈 {name} Patch Summary:")
        print(f"  Total patches analyzed: {total_patches}")
        print(f"  Patches with positives: {patches_with_positives} ({patches_with_positives/total_patches*100:.1f}%)")
        print(f"  Overall positive ratio: {overall_pos_ratio:.6f} ({overall_pos_ratio*100:.4f}%)")

        return overall_pos_ratio

    def check_data_sanity(self, max_batches: int = 3):
        """Comprehensive data sanity checks for all loaders"""
        print("\n" + "="*60)
        print("🔍 RUNNING COMPREHENSIVE DATA SANITY CHECKS")
        print("="*60)

        # Check training data
        print("\n📊 TRAINING DATA SANITY CHECK:")
        self._check_loader_sanity(self.train_loader, "train", max_batches)

        # Check validation data
        print("\n📊 VALIDATION DATA SANITY CHECK:")
        self._check_loader_sanity(self.valid_loader, "valid", max_batches)

        # Check test data
        print("\n📊 TEST DATA SANITY CHECK:")
        self._check_loader_sanity(self.test_loader, "test", max_batches)

        # Check full volumes (before patch sampling)
        print("\n📊 FULL VOLUME SANITY CHECK (Before Patch Sampling):")
        self._check_loader_sanity(self.full_train_loader, "full_train", max_batches)

    def _check_loader_sanity(self, loader, name: str, max_batches: int):
        """Check a specific loader for data sanity"""
        print(f"🔍 Checking {name} loader...")

        batch_count = 0
        total_positives = 0
        total_voxels = 0

        for i, batch in enumerate(loader):
            if i >= max_batches:
                break

            vol = batch["vol"]
            seg = batch["seg"]

            print(f"\n  Batch {i}:")
            print(f"    Volume shape: {vol.shape}")
            print(f"    Volume range: [{vol.min():.4f}, {vol.max():.4f}]")
            print(f"    Volume mean: {vol.mean():.4f} ± {vol.std():.4f}")
            print(f"    Segmentation shape: {seg.shape}")
            print(f"    Segmentation unique values: {torch.unique(seg)}")

            # Check for NaNs or Infs
            if torch.isnan(vol).any():
                print("    ⚠️ WARNING: Volume contains NaN values!")
            if torch.isinf(vol).any():
                print("    ⚠️ WARNING: Volume contains Inf values!")

            # Check segmentation values are binary
            unique_vals = torch.unique(seg)
            if not all(val in [0, 1] for val in unique_vals):
                print(f"    ⚠️ WARNING: Segmentation has non-binary values: {unique_vals}")

            # Calculate positives
            batch_positives = seg.sum().item()
            batch_voxels = seg.numel()
            pos_ratio = batch_positives / batch_voxels if batch_voxels > 0 else 0

            print(f"    Positives: {batch_positives:,}/{batch_voxels:,} ({pos_ratio*100:.4f}%)")

            total_positives += batch_positives
            total_voxels += batch_voxels
            batch_count += 1

        # Summary for this loader
        if batch_count > 0:
            overall_ratio = total_positives / total_voxels if total_voxels > 0 else 0
            print(f"\n  📈 {name} Summary ({batch_count} batches):")
            print(f"    Total positives: {total_positives:,}")
            print(f"    Total voxels: {total_voxels:,}")
            print(f"    Overall positive ratio: {overall_ratio:.6f} ({overall_ratio*100:.4f}%)")

    def check_data_distributions(self):
        """Check intensity distributions and data consistency"""
        print("\n" + "="*60)
        print("📊 CHECKING DATA DISTRIBUTIONS")
        print("="*60)

        # Check intensity distributions
        self._check_intensity_distribution(self.train_loader, "Training")
        self._check_intensity_distribution(self.valid_loader, "Validation")
        self._check_intensity_distribution(self.test_loader, "Test")

    def _check_intensity_distribution(self, loader, name: str):
        """Check intensity distribution for a loader"""
        print(f"\n📈 {name} Intensity Distribution:")

        all_values = []
        for i, batch in enumerate(loader):
            if i >= 5:  # Check first 5 batches for speed
                break
            vol = batch["vol"]
            all_values.extend(vol.flatten().cpu().numpy())

        if all_values:
            all_values = np.array(all_values)
            print(f"  Range: [{np.min(all_values):.4f}, {np.max(all_values):.4f}]")
            print(f"  Mean: {np.mean(all_values):.4f} ± {np.std(all_values):.4f}")
            print(f"  Median: {np.median(all_values):.4f}")
            print(f"  Non-zero values: {np.sum(all_values != 0):,}/{len(all_values):,} "
                  f"({np.sum(all_values != 0)/len(all_values)*100:.2f}%)")

    def check_spatial_consistency(self):
        """Check that all volumes have consistent spatial properties"""
        print("\n" + "="*60)
        print("📏 CHECKING SPATIAL CONSISTENCY")
        print("="*60)

        loaders = {
            "Training": self.train_loader,
            "Validation": self.valid_loader,
            "Test": self.test_loader
        }

        for name, loader in loaders.items():
            print(f"\n📐 {name} Spatial Properties:")
            shapes = []
            for i, batch in enumerate(loader):
                if i >= 3:  # Check first 3 batches
                    break
                vol = batch["vol"]
                shapes.append(vol.shape[2:])  # Get spatial dimensions (D, H, W)
                print(f"  Batch {i} shape: {vol.shape}")

            # Check consistency
            if len(shapes) > 1:
                if all(shape == shapes[0] for shape in shapes):
                    print(f"  ✅ All batches have consistent shape: {shapes[0]}")
                else:
                    print(f"  ⚠️ Shape inconsistency: {shapes}")

    def filter_empty_samples(self, file_list: List[Dict]) -> List[Dict]:
        """Remove samples that have no positive segmentation labels"""
        filtered_files = []

        for file_pair in file_list:
            # Load just the segmentation to check if it's empty
            seg_loader = Compose([
                LoadImaged(keys=["seg"]),
                EnsureChannelFirstd(keys=["seg"]),
            ])

            try:
                seg_data = seg_loader(file_pair)["seg"]
                if seg_data.sum() > 0:  # Only keep if has positive voxels
                    filtered_files.append(file_pair)
            except Exception as e:
                print(f"⚠️ Error loading {file_pair['seg']}: {e}")
                continue

        print(f"📊 Filtered {len(file_list)} -> {len(filtered_files)} non-empty samples")
        return filtered_files

    def prepare_loaders(self) -> Tuple[DataLoader, DataLoader, DataLoader, DataLoader]:
        """Prepare data loaders for training, validation, and testing"""
        # --- Constants ---
        pixdim = (1, 1, 1)
        a_min, a_max = -1000, 700
        patch_size = (512, 512, 512) # Target training patch size

        # NOTE: compatible_spatial_size is now redundant since we are targeting patch_size directly,
        # but we keep the name for validation/test sets which must be divisible by 16.
        compatible_spatial_size = (512, 512, 512)
        # -----------------

        def get_files(split: str) -> List[Dict]:
            """Get file paths for a specific split with validation (original helper, unchanged)"""
            ct_dir = os.path.join(self.dataset_dir, split, "ct")
            seg_dir = os.path.join(self.dataset_dir, split, "segment")

            # (File loading and error checks remain here)

            ct_files = sorted(glob(os.path.join(ct_dir, "*.nii.gz")))
            seg_files = sorted(glob(os.path.join(seg_dir, "*.nii.gz")))

            # (Error checks remain here)

            files = []
            for ct_file, seg_file in zip(ct_files, seg_files):
                ct_id = os.path.basename(ct_file).replace('.nii.gz', '')
                seg_id = os.path.basename(seg_file).replace('.nii.gz', '')
                if ct_id != seg_id:
                    print(f"⚠️ Warning: File name mismatch - CT: {ct_id}, SEG: {seg_id}")
                files.append({"vol": ct_file, "seg": seg_file})

            # Filter out empty samples for training
            if split == "train":
                files = self.filter_empty_samples(files)

            print(f"📁 {split}: Loaded {len(files)} samples")
            return files

        # ----------------------------------------------------------------------
        # Training transforms: Resize volume directly to 96x96x96
        # ----------------------------------------------------------------------
        train_transforms = Compose([
            LoadImaged(keys=["vol", "seg"]),
            EnsureChannelFirstd(keys=["vol", "seg"]),

            # FIX 1: Spacingd Interpolation (This is the one safe place for bilinear/nearest)
            Spacingd(keys=["vol", "seg"], pixdim=pixdim, mode=("bilinear", "nearest")),

            Orientationd(keys=["vol", "seg"], axcodes="RAS", labels=None),
            ScaleIntensityRanged(keys=["vol"], a_min=a_min, a_max=a_max, b_min=0.0, b_max=1.0, clip=True),
            NormalizeIntensityd(keys=["vol"], subtrahend=0.5, divisor=0.5),

            # FIX 2: Resize Interpolation/Padding (MUST be 'constant' to avoid crash)
            ResizeWithPadOrCropd(keys=["vol", "seg"],
                                spatial_size=patch_size,
                                # CRITICAL CHANGE: Use crash-proof CONSTANT for BOTH
                                mode=("constant", "constant"),
                                constant_values=(0.0, 0.0)),

            # Binarization
            Lambdad(keys="seg", func=binarize_segmentation_logic),

            RandFlipd(keys=["vol", "seg"], prob=0.5, spatial_axis=[0, 1, 2]),
            RandGaussianNoised(keys=["vol"], prob=0.2, mean=0.0, std=0.05),

            # Sampler (RandCropByPosNegLabeld must be here, NOT ResizeWithPadOrCropd)
            RandCropByPosNegLabeld(
                keys=["vol", "seg"],
                label_key="seg",
                spatial_size=patch_size,
                pos=0.9, neg=0.1, num_samples=8,
                image_key="vol", image_threshold=0, allow_smaller=False,
            ),

            ToTensord(keys=["vol", "seg"]),
            EnsureTyped(keys=["vol", "seg"]),
        ])

        # ----------------------------------------------------------------------
        # Base transforms: Use compatible_spatial_size (128^3) for evaluation/test
        # ----------------------------------------------------------------------
        base_transforms = Compose([
            LoadImaged(keys=["vol", "seg"]),
            EnsureChannelFirstd(keys=["vol", "seg"]),

            # FIX 1: Spacingd Interpolation (This is the one safe place for bilinear/nearest)
            Spacingd(keys=["vol", "seg"], pixdim=pixdim, mode=("bilinear", "nearest")),

            Orientationd(keys=["vol", "seg"], axcodes="RAS", labels=None),
            ScaleIntensityRanged(keys=["vol"], a_min=a_min, a_max=a_max, b_min=0.0, b_max=1.0, clip=True),
            NormalizeIntensityd(keys=["vol"], subtrahend=0.5, divisor=0.5),

            # FIX 2: Resize Interpolation/Padding (MUST be 'constant' to avoid crash)
            ResizeWithPadOrCropd(keys=["vol", "seg"],
                                spatial_size=patch_size,
                                # CRITICAL CHANGE: Use crash-proof CONSTANT for BOTH
                                mode=("constant", "constant"),
                                constant_values=(0.0, 0.0)),

            # CRITICAL FIX: Apply Lambdad with the global function (for picklability)
            Lambdad(keys="seg", func=binarize_segmentation_logic),

            ToTensord(keys=["vol", "seg"]),
            EnsureTyped(keys=["vol","seg"], track_meta=True),
        ])

        # Create datasets
        print("🔄 Loading training dataset...")
        train_ds = CacheDataset(
            data=get_files("train"),
            transform=train_transforms,
            cache_rate=0.1,
            num_workers=2 # <-- SPEED FIX: Re-enable multiprocessing for caching
        )

        print("🔄 Loading validation dataset...")
        valid_ds = CacheDataset(
            data=get_files("valid"),
            transform=base_transforms,
            cache_rate=0.2,
            num_workers=2 # <-- SPEED FIX
        )

        print("🔄 Loading test dataset...")
        test_ds = CacheDataset(
            data=get_files("test"),
            transform=base_transforms,
            cache_rate=0.2,
            num_workers=2 # <-- SPEED FIX
        )

        # Create data loaders with optimized settings
        train_loader = DataLoader(
            train_ds,
            batch_size=self.config['batch_size'],
            shuffle=True,
            num_workers=0, # <-- CRITICAL TEMPORARY FIX: Disable workers for training stability
            pin_memory=torch.cuda.is_available(),
            persistent_workers=False,
            # REMOVED prefetch_factor=2, # MUST be removed if num_workers=0
            collate_fn=pad_list_data_collate,
            drop_last=True
        )

        valid_loader = DataLoader(
            valid_ds,
            batch_size=self.config['batch_size'],
            shuffle=False,
            num_workers=0,  # Synchronous evaluation/analysis
            pin_memory=torch.cuda.is_available(),
            # Removed persistent_workers=False,
            # Removed prefetch_factor=1,
            collate_fn=pad_list_data_collate
        )

        test_loader = DataLoader(
            test_ds,
            batch_size=1,
            shuffle=False,
            num_workers=0,  # Synchronous evaluation/analysis
            pin_memory=False,
            # Removed persistent_workers=False,
            # Removed prefetch_factor=None,
            collate_fn=pad_list_data_collate
        )

        # Full volume train loader for sanity checks (no patching)
        full_train_ds = Dataset(get_files("train"), base_transforms)
        full_train_loader = DataLoader(
            full_train_ds,
            batch_size=1,
            shuffle=False,
            num_workers=0,  # Synchronous evaluation/analysis
            pin_memory=False,
            collate_fn=pad_list_data_collate
        )

        print(f"✅ Dataset sizes — Train: {len(train_ds)}, Valid: {len(valid_ds)}, Test: {len(test_ds)}")

        return train_loader, valid_loader, test_loader, full_train_loader


    def train(self):
        """Execute training with resource monitoring"""
        model = self.create_unet_model(self.device)
        # Monitor resources
        if torch.cuda.is_available():
            print(f"🎯 GPU Memory allocated: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
            print(f"🎯 GPU Memory cached: {torch.cuda.memory_reserved() / 1024**3:.2f} GB")

        # print("\n🔍 Final shape check before training...")
        # self.debug_model_shapes()
        # Validate we have data

        if len(self.train_loader.dataset) == 0:
            raise ValueError("No training data available!")
        if len(self.valid_loader.dataset) == 0:
            raise ValueError("No validation data available!")

        trainer = UnetTrain(
            model_file=self.model_file,
            loss_result_path=self.loss_result_file,
            lr=self.config['learning_rate'],
            num_epochs=self.config['num_epochs'],
            device=self.device
        )
        trainer.execute(model,self.train_loader, self.valid_loader)

    def test(self):
        """Execute testing with checkpoint validation"""
        # Use consistent architecture with training
        model = self.create_unet_model(self.device)

        if os.path.exists(self.model_file):
            print("📂 Loading checkpoint...")
            checkpoint = torch.load(self.model_file, map_location=self.device)

            # Validate checkpoint
            if 'model_state_dict' not in checkpoint:
                raise ValueError("Invalid checkpoint: missing 'model_state_dict'")

            try:
                model.load_state_dict(checkpoint['model_state_dict'])
                print("✅ Checkpoint loaded successfully")

                # Print training info if available
                if 'epoch' in checkpoint:
                    print(f"📅 Checkpoint from epoch: {checkpoint['epoch']}")
                if 'val_loss' in checkpoint:
                    print(f"📉 Checkpoint validation loss: {checkpoint['val_loss']:.6f}")

            except Exception as e:
                print(f"❌ Checkpoint loading failed: {e}")
                raise
        else:
            raise FileNotFoundError(f"Model file not found: {self.model_file}")

        tester = UnetTest(self.test_result_path, self.test_metrics_file, self.device)
        tester.test(model, self.test_loader)

    def run(self):
        """Execute full pipeline with validation"""
        print("🚀 Starting pipeline with:")
        print(f"   - Train samples: {len(self.train_loader.dataset)}")
        print(f"   - Valid samples: {len(self.valid_loader.dataset)}")
        print(f"   - Test samples: {len(self.test_loader.dataset)}")
        print(f"   - Device: {self.device}")
        print(f"   - Output directory: {self.output_dir}")

        start_time = time.time()

        try:
            self.train()
            self.test()

            total_time = time.time() - start_time
            print(f"✅ Pipeline completed in {total_time/60:.2f} minutes")

        except Exception as e:
            print(f"❌ Pipeline failed: {e}")
            raise


# ---------- Run ----------
def main():
    import multiprocessing as mp
    mp.set_start_method('spawn', force=True)

    config = {
        'target_dir': "/content/drive/MyDrive/PhDwork/Segmentation",
        'output_folder_name': "Results_Nifti_MONAI6_1_3Original",
        'transformation': "OriginalCT_Nifti_Empty_NonEmpty_slices_In_Train",
        'batch_size': 1,   # Effective patches per step = num_samples * batch_size
        'num_epochs': 200,
        'learning_rate': 1e-5,
    }

    pipeline = UnetPipeline(config)
    pipeline.run()

if __name__ == "__main__":
    main()

📁 Current Directory: /content/drive/MyDrive/PhDwork/Segmentation
📦 Preparing datasets and transforms...
🔄 Loading training dataset...
📊 Filtered 340 -> 340 non-empty samples
📁 train: Loaded 340 samples


Loading dataset: 100%|██████████| 34/34 [03:51<00:00,  6.81s/it]


🔄 Loading validation dataset...
📁 valid: Loaded 43 samples


Loading dataset: 100%|██████████| 8/8 [01:07<00:00,  8.45s/it]


🔄 Loading test dataset...
📁 test: Loaded 38 samples


Loading dataset: 100%|██████████| 7/7 [00:52<00:00,  7.45s/it]


📊 Filtered 340 -> 340 non-empty samples
📁 train: Loaded 340 samples
✅ Dataset sizes — Train: 340, Valid: 43, Test: 38
🚀 Starting pipeline with:
   - Train samples: 340
   - Valid samples: 43
   - Test samples: 38
   - Device: cuda
   - Output directory: ./results/Results_Nifti_MONAI6_1_3Original
⚠️ DynamicUNet not available or import failed. Using SafeUNet wrapper.
🎯 GPU Memory allocated: 0.07 GB
🎯 GPU Memory cached: 0.08 GB
Batch 0: pos=97992.0, neg=1073643832.0, pos_ratio=0.000091
Batch 1: pos=206760.0, neg=1073535064.0, pos_ratio=0.000193
Batch 2: pos=145848.0, neg=1073595976.0, pos_ratio=0.000136
Batch 3: pos=29880.0, neg=1073711944.0, pos_ratio=0.000028
Batch 4: pos=40968.0, neg=1073700856.0, pos_ratio=0.000038
Batch 5: pos=265080.0, neg=1073476744.0, pos_ratio=0.000247
Batch 6: pos=1733768.0, neg=1072008056.0, pos_ratio=0.001615
Batch 7: pos=91992.0, neg=1073649832.0, pos_ratio=0.000086
Batch 8: pos=1619512.0, neg=1072122312.0, pos_ratio=0.001508
Batch 9: pos=20664.0, neg=1073721

OutOfMemoryError: CUDA out of memory. Tried to allocate 8.00 GiB. GPU 0 has a total capacity of 79.32 GiB of which 727.88 MiB is free. Process 24588 has 78.57 GiB memory in use. Of the allocated memory 76.07 GiB is allocated by PyTorch, and 2.01 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

#(8) Mask Generation

In [ ]:
import os
import torch
import nibabel as nib
import numpy as np
from pathlib import Path
from monai.networks.nets import UNet
from monai.transforms import (
    Compose,
    Resized,
    CopyItemsd,
    Invertd,
    LoadImaged,
    EnsureChannelFirstd,
    Spacingd,
    Orientationd,
    ScaleIntensityRanged,
    CropForegroundd,
    EnsureTyped,
    SaveImaged,
    ToTensord,
)
from monai.data import Dataset, DataLoader, decollate_batch
from monai.inferers import sliding_window_inference
from monai.utils import set_determinism
from monai.networks.layers import Norm
# from monai.transforms.utils import SaveTransform



class UNetInferencePipeline:
    def __init__(self, model_path, input_ct_dir, input_seg_dir, output_dir, device="cuda:0"):
        self.device = device if torch.cuda.is_available() else "cpu"
        self.input_ct_dir = input_ct_dir
        self.input_seg_dir = input_seg_dir
        self.output_dir = output_dir
        self.ct_out_dir = os.path.join(output_dir, "ct")
        self.seg_out_dir = os.path.join(output_dir, "segment")
        os.makedirs(self.ct_out_dir, exist_ok=True)
        os.makedirs(self.seg_out_dir, exist_ok=True)
        self.model_path = model_path
        self.model = self._load_model()
        set_determinism(seed=42)
        self.forward_transforms = self._get_forward_transforms()
        self.inverse_transforms = None
        self.dataloader = self._prepare_dataloader()

    def _load_model(self):
        if not os.path.exists(self.model_path):
            raise FileNotFoundError(f"Model file not found at: {self.model_path}")

        model = UNet(
            spatial_dims=3,
            in_channels=1,
            out_channels=1,
            channels=(16, 32, 64, 128, 256),
            strides=(2, 2, 2, 2),
            num_res_units=2,
            norm=Norm.BATCH
        ).to(self.device)

        state_dict = torch.load(self.model_path, map_location=self.device)
        model.load_state_dict(state_dict.get('model_state_dict', state_dict))

        print(f"✅ Model loaded successfully from {self.model_path}")
        return model



    def _get_forward_transforms(self):
        return Compose([
            LoadImaged(keys=["vol"]),
            EnsureChannelFirstd(keys=["vol"]),
            CopyItemsd(keys=["vol"], names=["vol_meta_dict"]),
            Spacingd(keys=["vol"], pixdim=(1.0, 1.0, 1.0), mode="bilinear"),
            Orientationd(keys=["vol"], axcodes="RAS"),
            ScaleIntensityRanged(keys=["vol"], a_min=-1000, a_max=700, b_min=0.0, b_max=1.0, clip=True),
            CropForegroundd(keys=["vol"], source_key="vol"),
            Resized(keys=["vol"], spatial_size=(96, 96, 96)),
            EnsureTyped(keys=["vol"]),
        ])

    def _get_inverse_transforms(self):
        return Compose([
            Invertd(
                keys=["seg"],
                transform=self.forward_transforms,
                orig_keys=["vol"],
                meta_keys=["vol_meta_dict"],
                nearest_interp=True,
                to_tensor=False,
            ),
            EnsureTyped(keys=["seg"])
        ])

    def _prepare_dataloader(self):
        data = []
        for f in os.listdir(self.input_ct_dir):
            if f.endswith(('.nii', '.nii.gz')):
                ct_path = os.path.join(self.input_ct_dir, f)
                data.append({"vol": ct_path})
        print(f"🔍 Found {len(data)} NIfTI files for inference.")
        return DataLoader(Dataset(data=data, transform=self.forward_transforms), batch_size=1, num_workers=0)

    def infer(self):
        self.model.eval()
        with torch.no_grad():
            for i, batch in enumerate(self.dataloader):
                batch = decollate_batch(batch)[0]
                vol_meta = batch["vol_meta_dict"]
                ct = batch["vol"]

                if ct.dim() == 4:
                    ct = ct.unsqueeze(0)
                ct = ct.to(self.device)

                filename = os.path.basename(vol_meta.meta["filename_or_obj"])
                orig_vol = nib.load(vol_meta.meta["filename_or_obj"]).get_fdata()
                print(f"🔍 Inference on [{i+1}] {filename} | shape = {ct.shape}")
                print(f"🔍 Original volume shape = {orig_vol.shape}")
                pred = self.model(ct)
                pred = (torch.sigmoid(pred) > 0.5).float()

                print(f"✅ Predicted mask shape: {pred.shape}")

                batch["seg"] = pred.cpu().squeeze(0)
                print(f"✅ Batch shape: {batch['seg'].shape}")

                if self.inverse_transforms is None:
                    self.inverse_transforms = self._get_inverse_transforms()

                inverted = self.inverse_transforms(batch)
                inv_seg = inverted["seg"].squeeze(0).numpy()
                inv_seg = (inv_seg > 0.5).astype(np.uint8)
                print(f"✅ Inverted mask shape: {inv_seg.shape}")

                self._save_nifti(inv_seg, vol_meta, self.seg_out_dir, filename, is_segmentation=True)


    def _save_nifti(self, array, meta_tensor, out_dir, filename, is_segmentation=False):
        os.makedirs(out_dir, exist_ok=True)
        affine = meta_tensor.meta.get("original_affine", meta_tensor.meta.get("affine", np.eye(4)))
        dtype = np.uint8 if is_segmentation else np.float32
        nib_img = nib.Nifti1Image(array.astype(dtype), affine)
        nib.save(nib_img, os.path.join(out_dir, filename))
        print(f"✅ Saved: {os.path.join(out_dir, filename)}")


if __name__ == "__main__":
    ROOT_DIR = "/content/drive/MyDrive/PhDwork/Segmentation"
    MODEL_PATH = os.path.join(ROOT_DIR, "results", "Results_MONAI_Augmented", "model.pth")
    INPUT_CT_FOLDER = os.path.join(ROOT_DIR, "datasets", "Datasets_OriginalCT_Nifti_Empty_NonEmpty_slices_In_Train", "Lung3", "ct")
    INPUT_SEG_FOLDER = os.path.join(ROOT_DIR, "datasets", "Datasets_OriginalCT_Nifti_Empty_NonEmpty_slices_In_Train", "Lung3", "segment")
    OUTPUT_FOLDER = os.path.join(ROOT_DIR, "datasets", "Datasets_OriginalCT_Nifti_Empty_NonEmpty_slices_In_Train", "Lung3_Predicted")

    os.makedirs(OUTPUT_FOLDER, exist_ok=True)
    os.chdir(ROOT_DIR)

    try:
        pipeline = UNetInferencePipeline(MODEL_PATH, INPUT_CT_FOLDER, INPUT_SEG_FOLDER, OUTPUT_FOLDER)
        pipeline.infer()
        print("🎉 Inference completed successfully for all patients!")
    except FileNotFoundError as e:
        print(f"Error: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")


✅ Model loaded successfully from /content/drive/MyDrive/PhDwork/Segmentation/results/Results_MONAI_Augmented/model.pth
🔍 Found 89 NIfTI files for inference.
🔍 Inference on [1] LUNG3-01.nii.gz | shape = torch.Size([1, 1, 96, 96, 96])
🔍 Original volume shape = (59, 512, 512)
✅ Predicted mask shape: torch.Size([1, 1, 96, 96, 96])
✅ Batch shape: torch.Size([1, 96, 96, 96])
✅ Inverted mask shape: (59, 512, 512)
✅ Saved: /content/drive/MyDrive/PhDwork/Segmentation/datasets/Datasets_OriginalCT_Nifti_Empty_NonEmpty_slices_In_Train/Lung3_Predicted/segment/LUNG3-01.nii.gz
🔍 Inference on [2] LUNG3-02.nii.gz | shape = torch.Size([1, 1, 96, 96, 96])
🔍 Original volume shape = (57, 512, 512)
✅ Predicted mask shape: torch.Size([1, 1, 96, 96, 96])
✅ Batch shape: torch.Size([1, 96, 96, 96])
✅ Inverted mask shape: (57, 512, 512)
✅ Saved: /content/drive/MyDrive/PhDwork/Segmentation/datasets/Datasets_OriginalCT_Nifti_Empty_NonEmpty_slices_In_Train/Lung3_Predicted/segment/LUNG3-02.nii.gz
🔍 Inference on [3] 

In [ ]:
if __name__ == "__main__":
    ROOT_DIR = "/content/drive/MyDrive/PhDwork/Segmentation"
    MODEL_PATH = os.path.join(ROOT_DIR, "results", "Results_MONAI_Augmented", "model.pth")
    INPUT_CT_FOLDER = os.path.join(ROOT_DIR, "datasets", "Datasets_OriginalCT_Nifti_Empty_NonEmpty_slices_In_Train", "test", "ct")
    INPUT_SEG_FOLDER = os.path.join(ROOT_DIR, "datasets", "Datasets_OriginalCT_Nifti_Empty_NonEmpty_slices_In_Train", "test", "segment")
    OUTPUT_FOLDER = os.path.join(ROOT_DIR, "datasets", "Datasets_OriginalCT_Nifti_Empty_NonEmpty_slices_In_Train", "test_Predicted")

    os.makedirs(OUTPUT_FOLDER, exist_ok=True)
    os.chdir(ROOT_DIR)

    try:
        pipeline = UNetInferencePipeline(MODEL_PATH, INPUT_CT_FOLDER, INPUT_SEG_FOLDER, OUTPUT_FOLDER)
        pipeline.infer()
        print("🎉 Inference completed successfully for all patients!")
    except FileNotFoundError as e:
        print(f"Error: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

✅ Model loaded successfully from /content/drive/MyDrive/PhDwork/Segmentation/results/Results_MONAI_Augmented/model.pth
🔍 Found 38 NIfTI files for inference.
🔍 Inference on [1] LUNG1-001.nii.gz | shape = torch.Size([1, 1, 96, 96, 96])
🔍 Original volume shape = (134, 512, 512)
✅ Predicted mask shape: torch.Size([1, 1, 96, 96, 96])
✅ Batch shape: torch.Size([1, 96, 96, 96])
✅ Inverted mask shape: (134, 512, 512)
✅ Saved: /content/drive/MyDrive/PhDwork/Segmentation/datasets/Datasets_OriginalCT_Nifti_Empty_NonEmpty_slices_In_Train/test_Predicted/segment/LUNG1-001.nii.gz
🔍 Inference on [2] LUNG1-025.nii.gz | shape = torch.Size([1, 1, 96, 96, 96])
🔍 Original volume shape = (106, 512, 512)
✅ Predicted mask shape: torch.Size([1, 1, 96, 96, 96])
✅ Batch shape: torch.Size([1, 96, 96, 96])
✅ Inverted mask shape: (106, 512, 512)
✅ Saved: /content/drive/MyDrive/PhDwork/Segmentation/datasets/Datasets_OriginalCT_Nifti_Empty_NonEmpty_slices_In_Train/test_Predicted/segment/LUNG1-025.nii.gz
🔍 Inference o

In [ ]:
if __name__ == "__main__":
    ROOT_DIR = "/content/drive/MyDrive/PhDwork/Segmentation"
    MODEL_PATH = os.path.join(ROOT_DIR, "results", "Results_MONAI_Augmented", "model.pth")
    INPUT_CT_FOLDER = os.path.join(ROOT_DIR, "datasets", "Datasets_OriginalCT_Nifti_Empty_NonEmpty_slices_In_Train", "valid", "ct")
    INPUT_SEG_FOLDER = os.path.join(ROOT_DIR, "datasets", "Datasets_OriginalCT_Nifti_Empty_NonEmpty_slices_In_Train", "valid", "segment")
    OUTPUT_FOLDER = os.path.join(ROOT_DIR, "datasets", "Datasets_OriginalCT_Nifti_Empty_NonEmpty_slices_In_Train", "valid_Predicted")

    os.makedirs(OUTPUT_FOLDER, exist_ok=True)
    os.chdir(ROOT_DIR)

    try:
        pipeline = UNetInferencePipeline(MODEL_PATH, INPUT_CT_FOLDER, INPUT_SEG_FOLDER, OUTPUT_FOLDER)
        pipeline.infer()
        print("🎉 Inference completed successfully for all patients!")
    except FileNotFoundError as e:
        print(f"Error: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

✅ Model loaded successfully from /content/drive/MyDrive/PhDwork/Segmentation/results/Results_MONAI_Augmented/model.pth
🔍 Found 43 NIfTI files for inference.
🔍 Inference on [1] LUNG1-010.nii.gz | shape = torch.Size([1, 1, 96, 96, 96])
🔍 Original volume shape = (91, 512, 512)
✅ Predicted mask shape: torch.Size([1, 1, 96, 96, 96])
✅ Batch shape: torch.Size([1, 96, 96, 96])
✅ Inverted mask shape: (91, 512, 512)
✅ Saved: /content/drive/MyDrive/PhDwork/Segmentation/datasets/Datasets_OriginalCT_Nifti_Empty_NonEmpty_slices_In_Train/valid_Predicted/segment/LUNG1-010.nii.gz
🔍 Inference on [2] LUNG1-031.nii.gz | shape = torch.Size([1, 1, 96, 96, 96])
🔍 Original volume shape = (153, 512, 512)
✅ Predicted mask shape: torch.Size([1, 1, 96, 96, 96])
✅ Batch shape: torch.Size([1, 96, 96, 96])
✅ Inverted mask shape: (153, 512, 512)
✅ Saved: /content/drive/MyDrive/PhDwork/Segmentation/datasets/Datasets_OriginalCT_Nifti_Empty_NonEmpty_slices_In_Train/valid_Predicted/segment/LUNG1-031.nii.gz
🔍 Inference o

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from typing import List
import os
import csv


class LossPlotter:
    def __init__(self, csv_path: str):
        self.csv_path = Path(csv_path)
        self.data = self._load_data()

    def _load_data(self):
        if not self.csv_path.exists():
            raise FileNotFoundError(f"CSV file not found: {self.csv_path}")
        df = pd.read_csv(self.csv_path, index_col=0)  # Read row labels as index
        return df  # Make rows into columns

    def plot(self, title: str = "Training and Validation Loss", save_path= None):
        plt.figure(figsize=(8, 5))
        plt.plot(self.data.index, self.data['Train Loss'], label='Train Loss', color='blue')
        plt.plot(self.data.index, self.data['Valid Loss'], label='Valid Loss', color='orange')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.title(title)
        plt.legend()
        plt.grid(True)
        plt.tight_layout()

        if save_path:
            save_path = Path(save_path)
            save_path.parent.mkdir(parents=True, exist_ok=True)
            plt.savefig(save_path, format='pdf')
            print(f"[INFO] Loss plot saved to {save_path}")
        else:
            plt.show()

        plt.close()

if __name__ == "__main__":
    target_dir = "/content/drive/MyDrive/PhDwork/Segmentation"
    os.chdir(target_dir)
    loss_result_file = os.path.join(".","results",f"Results_PreProcessedCT_Fifty_Fifty_DiceLoss_And_Strong_Augmentation","train_and_valid_loss_results.csv")
    plotter = LossPlotter(loss_result_file)
    plotter.plot()


In [ ]:
import h5py
os.chdir("/content/drive/MyDrive/PhDwork/Segmentation")
print(f"📁 Current Directory: {os.getcwd()}")
with h5py.File('./datasets/Datasets_PreprocessedCT_clipping_uniformSpacing_With_Empty_NonEmpty_slices_In_Train/train_dataset.hdf5', 'r') as f:
    print(list(f.keys()))